# Leveraging HPC With Azure Machine Learning Service

Azure Machine Learning service is a cloud service that you can use to develop and deploy machine learning models. Using Azure Machine Learning service, you can track your models as you build, train, deploy, and manage them, all at the broad scale that the cloud provides.
![](./images/aml-overview.png)

### Connect To Workspace

In [1]:
from azureml.core import Workspace

workspace = Workspace.from_config()

### Create Compute Target

In [2]:
from azureml.core.compute import AmlCompute, ComputeTarget

cluster_name = 'p100cluster'
compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_NC12s_v2', 
                                                       idle_seconds_before_scaledown=6000,
                                                       min_nodes=0, 
                                                       max_nodes=4)

compute_target = ComputeTarget.create(workspace, cluster_name, compute_config)
compute_target.wait_for_completion(show_output=True)

Succeeded
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned


### Submit Job

In [6]:
from azureml.core import Experiment

experiment_name = 'bert-mrpc' 
experiment = Experiment(workspace, name=experiment_name)

In [49]:
from azureml.train.dnn import TensorFlow

estimator = TensorFlow(source_directory='.',
                      entry_script='train.py',
                      compute_target=compute_target,
                      framework_version='2.0',
                      use_gpu=True,
                      pip_packages=['transformers', 'tensorflow_datasets'])

run = experiment.submit(estimator)

In [51]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

### Register and Deploy Model

In [52]:
model = run.register_model(model_name='bert-mrpc', model_path='outputs', description='BERT model trained on MRPC dataset')

In [90]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(source_directory='.',
                                   runtime='python', 
                                   entry_script='score.py',
                                   conda_file='env.yml')

In [97]:
model = Model(workspace, name='bert-mrpc')

In [119]:
from azureml.core.webservice import Webservice
from azureml.exceptions import WebserviceException
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core import Model

aci_config = AciWebservice.deploy_configuration(cpu_cores=2, 
                                                memory_gb=4, 
                                                tags={'model': 'BERT', 'method': 'tensorflow'}, 
                                                description='Predicts on MRPC data')

aci_service_name = 'bert-mrpc-aciservice'

try:
    aci_service = Webservice(workspace, name=aci_service_name)
    if aci_service:
        aci_service.delete()
except WebserviceException as e:
    print()

aci_service = Model.deploy(workspace, aci_service_name, [model], inference_config, aci_config)
aci_service.wait_for_deployment(True)

Running.................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


### Test Service

In [120]:
aci_service.scoring_uri

'http://db84a2ea-4552-44f1-b9e9-f3a9f0336317.southcentralus.azurecontainer.io/score'

In [128]:
import requests 

sentence_0 = 'I want to go get a drink'
sentence_1 = 'I want to go get a coffee'

input_data = '{\"sentence_0\": \"'+ sentence_0 +'\", "sentence_1\": \"'+ sentence_1 +'\"}'
headers = {'Content-Type':'application/json'}
response = requests.post(aci_service.scoring_uri, input_data, headers=headers)
json.loads(response.text)['result']

" 'I want to go get a drink' is a paraphrase of 'I want to go get a coffee' "

In [126]:
sentence_0 = 'Pytorch is amazing'
sentence_1 = 'Tensorflow is amazing'

input_data = '{\"sentence_0\": \"'+ sentence_0 +'\", "sentence_1\": \"'+ sentence_1 +'\"}'
headers = {'Content-Type':'application/json'}
response = requests.post(aci_service.scoring_uri, input_data, headers=headers)
json.loads(response.text)['result']

" 'Pytorch is amazing' is not a paraphrase of 'Tensorflow is amazing' "